In [1]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from os.path import join as oj
import sys
import numpy as np
from copy import deepcopy

In [88]:
class StackNet(nn.Module):
    def __init__(self):
        super(StackNet, self).__init__()
        self.lin = nn.Linear(1376, 10)
        # input: (N, C_in, D, H, W)
        # output: (N, C_out, D_out, H_out, W_out)        
        self.lay = torch.nn.Conv3d(in_channels=1, out_channels=4, kernel_size=(3, 7, 7), 
                        stride=1, padding=0, dilation=1)
        
    def features(self, x):
        print('init shape', x.shape)
        for i in range(5):
            x = self.lay(x)
    #         x = nn.BatchNorm2d()(x)
            x = nn.ReLU()(x)
            print('pass', i,  'shape', x.shape)
            x = x.reshape(x.shape[0], 1, x.shape[1] * x.shape[2], x.shape[3], x.shape[4])
            print('convert back to', x.shape)
        return x
         
#         nn.BatchNorm2d(x),
#         nn.ReLU(inplace=True)


    def forward(self, x):
        y = self.features(x)
        y = y.reshape(y.shape[0], -1)
        y = self.lin(y)
        return y

model = StackNet()
x = torch.randn(2, 3, 32, 32, requires_grad=True)

# print(x.shape, type(x))
y = model(x)
print('out shape', y.shape)

init shape torch.Size([2, 1, 3, 32, 32])
pass 0 shape torch.Size([2, 4, 1, 26, 26])
convert back to torch.Size([2, 1, 4, 26, 26])
pass 1 shape torch.Size([2, 4, 2, 20, 20])
convert back to torch.Size([2, 1, 8, 20, 20])
pass 2 shape torch.Size([2, 4, 6, 14, 14])
convert back to torch.Size([2, 1, 24, 14, 14])
pass 3 shape torch.Size([2, 4, 22, 8, 8])
convert back to torch.Size([2, 1, 88, 8, 8])
pass 4 shape torch.Size([2, 4, 86, 2, 2])
convert back to torch.Size([2, 1, 344, 2, 2])
out shape torch.Size([2, 10])


In [67]:
print(model.state_dict()['lay.weight'].shape)

torch.Size([4, 1, 3, 7, 7])
